In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
from skimage import io, color, transform, feature
%matplotlib inline

In [2]:
df = pd.read_csv('/Volumes/Samsung_t3/project-repos/jfi/cashtransfers/data/data2_modified.csv')
df.head(2)

,Unnamed: 0,No.,Paper,Summary of findings,Category,URL,Country,Allocation,Nature of Cash Transfer,Countries involved,...,Psychological Effects,Social Effects,Health Effects,Effects on Schooling,Means-Testing,Research Design,Effects on Employment,cat1,cat2,cat3
0,0,1,"Forget, E. L. (2011). The town with no poverty...",In the Manitoba basic annual income field expe...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.utpjournals.press/doi/full/10.3138...,NaN,SB,A guaranteed annual income: Familities with no...,Canada,...,NaN,NaN,Y,NaN,NaN,NaN,NaN,0.0,1.0,0.0
1,1,2,"Forget, E. L., Peden, A., & Strobel, S. (2013)...","According to Forget, Peden and Strobel (2013),...",Universal Basic Income,http://citeseerx.ist.psu.edu/viewdoc/download?...,NaN,GT,NaN,NaN,...,NaN,Y,NaN,NaN,Y,NaN,NaN,0.0,0.0,1.0


In [3]:
list(df)

['Unnamed: 0',
 'No.',
 'Paper',
 'Summary of findings',
 'Category',
 'URL',
 'Country',
 'Allocation',
 'Nature of Cash Transfer',
 'Countries involved',
 'Useful Tools',
 'Relevance',
 'Economic Effects',
 'Psychological Effects',
 'Social Effects',
 'Health Effects',
 'Effects on Schooling',
 'Means-Testing',
 'Research Design',
 'Effects on Employment',
 'cat1',
 'cat2',
 'cat3']

In [4]:
df = df.drop('Unnamed: 0', 1)
df = df.drop('No.', 1)

In [5]:
# rename columns
df.columns = ['paper', 'summ', 'category','url', 'country', 'allocation', 'NOCT', 'involved', 'tools', 'relevance', 'economic', 'psych', 'social', 'health', 'schooling', 'means', 'research_design', 'employment_effects', 'cat1', 'cat2', 'cat3']

In [6]:
df = df.fillna("")
df['size'] = 40000

In [7]:
cols = ['economic', 'psych', 'social', 'health', 'schooling']
df[cols] = df[cols].replace({'Y':1, "":0})

In [8]:
# Original
df.head(3)

,paper,summ,category,url,country,allocation,NOCT,involved,tools,relevance,...,social,health,schooling,means,research_design,employment_effects,cat1,cat2,cat3,size
0,"Forget, E. L. (2011). The town with no poverty...",In the Manitoba basic annual income field expe...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.utpjournals.press/doi/full/10.3138...,,SB,A guaranteed annual income: Familities with no...,Canada,,"Effects on population health, reflected in eco...",...,0,1,0,,,,0.0,1.0,0.0,40000
1,"Forget, E. L., Peden, A., & Strobel, S. (2013)...","According to Forget, Peden and Strobel (2013),...",Universal Basic Income,http://citeseerx.ist.psu.edu/viewdoc/download?...,,GT,,,,Spillover effects of UBI,...,1,0,0,Y,,,0.0,0.0,1.0,40000
2,"A. Goodman-Bacon, L. McGranahan (2008). How Do...",The Earned Income Tax Credit (EITC) in the U.S...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.chicagofed.org/publications/econom...,,SB,Earned Income Tax Credit (EITC): a tax credit ...,U.S.,,"Economic effects, measurement and tracking of ...",...,0,0,0,,,,0.0,1.0,0.0,40000


In [9]:
#or set columns names by list
s = df[cols].stack()
df1 = pd.get_dummies(s[s == 1].reset_index(level=1).drop(0, axis=1), prefix='', prefix_sep='')

In [10]:
df2 = pd.merge(df, df1, left_index=True, right_index=True)
print(list(df2))
df2.shape

['paper', 'summ', 'category', 'url', 'country', 'allocation', 'NOCT', 'involved', 'tools', 'relevance', 'economic_x', 'psych_x', 'social_x', 'health_x', 'schooling_x', 'means', 'research_design', 'employment_effects', 'cat1', 'cat2', 'cat3', 'size', 'economic_y', 'health_y', 'psych_y', 'schooling_y', 'social_y']


(59, 27)

In [11]:
df2= df2.drop(['economic_x', 'psych_x', 'social_x', 'health_x', 'schooling_x'], axis=1)
df2.columns = ['paper', 'summ', 'category','url', 'country', 'allocation', 'NOCT', 'involved', 'tools', 'relevance', 'means', 'research_design', 'employment_effects', 'cat1', 'cat2', 'cat3', 'size', 'economic', 'health', 'psych', 'schooling', 'social']
df2

,paper,summ,category,url,country,allocation,NOCT,involved,tools,relevance,...,employment_effects,cat1,cat2,cat3,size,economic,health,psych,schooling,social
0,"Forget, E. L. (2011). The town with no poverty...",In the Manitoba basic annual income field expe...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.utpjournals.press/doi/full/10.3138...,,SB,A guaranteed annual income: Familities with no...,Canada,,"Effects on population health, reflected in eco...",...,,0.0,1.0,0.0,40000,0,1,0,0,0
1,"Forget, E. L., Peden, A., & Strobel, S. (2013)...","According to Forget, Peden and Strobel (2013),...",Universal Basic Income,http://citeseerx.ist.psu.edu/viewdoc/download?...,,GT,,,,Spillover effects of UBI,...,,0.0,0.0,1.0,40000,0,0,0,0,1
2,"A. Goodman-Bacon, L. McGranahan (2008). How Do...",The Earned Income Tax Credit (EITC) in the U.S...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.chicagofed.org/publications/econom...,,SB,Earned Income Tax Credit (EITC): a tax credit ...,U.S.,,"Economic effects, measurement and tracking of ...",...,,0.0,1.0,0.0,40000,1,0,0,0,0
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,Y,1.0,0.0,0.0,40000,1,0,0,0,0
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,Y,1.0,0.0,0.0,40000,0,0,1,0,0
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,Y,1.0,0.0,0.0,40000,0,0,0,0,1
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,Y,1.0,0.0,0.0,40000,0,1,0,0,0
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,Y,1.0,0.0,0.0,40000,0,0,0,1,0
7,"Hagen-Zanker, J., Bastagli, F., Harman, L., Ba...",A systematic review of 201 studies by the Over...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.odi.org/publications/10505-cash-tr...,,GT,A systematic review of 201 studies on cash tra...,,,"Economic, Health, Employment, Educational Effe...",...,Y,1.0,0.0,0.0,40000,1,0,0,0,0
7,"Hagen-Zanker, J., Bastagli, F., Harman, L., Ba...",A systematic review of 201 studies by the Over...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.odi.org/publications/10505-cash-tr...,,GT,A systematic review of 201 studies on cash tra...,,,"Economic, Health, Employment, Educational Effe...",...,Y,1.0,0.0,0.0,40000,0,1,0,0,0


In [12]:
df2.shape

(59, 22)

In [13]:
df3 = df[df[cols].sum(axis=1) == 1]

In [14]:
# Get all rows that required no duplicating
df3.head(5)

,paper,summ,category,url,country,allocation,NOCT,involved,tools,relevance,...,social,health,schooling,means,research_design,employment_effects,cat1,cat2,cat3,size
0,"Forget, E. L. (2011). The town with no poverty...",In the Manitoba basic annual income field expe...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.utpjournals.press/doi/full/10.3138...,,SB,A guaranteed annual income: Familities with no...,Canada,,"Effects on population health, reflected in eco...",...,0,1,0,,,,0.0,1.0,0.0,40000
1,"Forget, E. L., Peden, A., & Strobel, S. (2013)...","According to Forget, Peden and Strobel (2013),...",Universal Basic Income,http://citeseerx.ist.psu.edu/viewdoc/download?...,,GT,,,,Spillover effects of UBI,...,1,0,0,Y,,,0.0,0.0,1.0,40000
2,"A. Goodman-Bacon, L. McGranahan (2008). How Do...",The Earned Income Tax Credit (EITC) in the U.S...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.chicagofed.org/publications/econom...,,SB,Earned Income Tax Credit (EITC): a tax credit ...,U.S.,,"Economic effects, measurement and tracking of ...",...,0,0,0,,,,0.0,1.0,0.0,40000
8,"H. Bewley, A. Bryson, J. Forth. Small and Med...","In Great Britain, employees at small firms are...",,http://www.psi.org.uk/site/publication_detail/...,,-,,U.K.,Variables and measures of employee well-being ...,Effects on workplace relations and employee-we...,...,0,0,0,,,Y,0.0,0.0,0.0,40000
9,John W. Budd. Worker Well-being and the Import...,Worker well-being is optimized when an individ...,,http://journals.sagepub.com/doi/abs/10.1177/09...,,-,,Germany,German Socioeconomic Panel (GSOEP),Psychological Effects,...,0,0,0,,,Y,0.0,0.0,0.0,40000


In [15]:
# Join subsets
df4 = pd.concat([df2,df3])
df4 = df4.fillna("")

In [16]:
list(df4)

['NOCT',
 'allocation',
 'cat1',
 'cat2',
 'cat3',
 'category',
 'country',
 'economic',
 'employment_effects',
 'health',
 'involved',
 'means',
 'paper',
 'psych',
 'relevance',
 'research_design',
 'schooling',
 'size',
 'social',
 'summ',
 'tools',
 'url']

In [17]:
# Reorder columns
col_order = ['paper', 'summ', 'category', 'url', 'country', 'allocation', 'NOCT', 'involved', 'tools', 'relevance', 'economic', 'psych', 'social', 'health', 'schooling', 'means', 'research_design', 'employment_effects', 'cat1', 'cat2', 'cat3','size']
df4 = df4[col_order]

In [18]:
# roots
df4.loc[df4['cat1']==1, "cat_recode"] = "1"
df4.loc[df4['cat2']==1, "cat_recode"] = "2"
df4.loc[df4['cat3']==1, "cat_recode"] = "3"

# icons
df4.loc[df4['economic']==1, "path"] = "4"
df4.loc[df4['psych']==1, "path"] = "4"
df4.loc[df4['social']==1, "path"] = "6"
df4.loc[df4['health']==1, "path"] = "7"
df4.loc[df4['schooling']==1, "path"] = "8"

# hover icons
df4.loc[df4['economic']==1, "hover"] = "4_hover"
df4.loc[df4['psych']==1, "hover"] = "5_hover"
df4.loc[df4['social']==1, "hover"] = "6_hover"
df4.loc[df4['health']==1, "hover"] = "7_hover"
df4.loc[df4['schooling']==1, "hover"] = "8_hover"

In [19]:
df4 = df4.drop_duplicates(subset=['paper', 'path'], keep="first")

In [20]:
df4

,paper,summ,category,url,country,allocation,NOCT,involved,tools,relevance,...,means,research_design,employment_effects,cat1,cat2,cat3,size,cat_recode,path,hover
0,"Forget, E. L. (2011). The town with no poverty...",In the Manitoba basic annual income field expe...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.utpjournals.press/doi/full/10.3138...,,SB,A guaranteed annual income: Familities with no...,Canada,,"Effects on population health, reflected in eco...",...,,,,0.0,1.0,0.0,40000,2,7,7_hover
1,"Forget, E. L., Peden, A., & Strobel, S. (2013)...","According to Forget, Peden and Strobel (2013),...",Universal Basic Income,http://citeseerx.ist.psu.edu/viewdoc/download?...,,GT,,,,Spillover effects of UBI,...,Y,,,0.0,0.0,1.0,40000,3,6,6_hover
2,"A. Goodman-Bacon, L. McGranahan (2008). How Do...",The Earned Income Tax Credit (EITC) in the U.S...,"Earned Income, Earned Income Tax Credit and Ne...",https://www.chicagofed.org/publications/econom...,,SB,Earned Income Tax Credit (EITC): a tax credit ...,U.S.,,"Economic effects, measurement and tracking of ...",...,,,,0.0,1.0,0.0,40000,2,4,4_hover
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,,,Y,1.0,0.0,0.0,40000,1,4,4_hover
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,,,Y,1.0,0.0,0.0,40000,1,6,6_hover
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,,,Y,1.0,0.0,0.0,40000,1,7,7_hover
3,"Haushofer, J., & Shapiro, J. (2013). Household...",Using a two stage randomization design in a ra...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.princeton.edu/~joha/publications/H...,,GT,RCT: Selection of treatment group was randomi...,Kenya,"Surveys on consumption, asset holdings, employ...","Economic, Health, Employment, Social, Educatio...",...,,,Y,1.0,0.0,0.0,40000,1,8,8_hover
7,"Hagen-Zanker, J., Bastagli, F., Harman, L., Ba...",A systematic review of 201 studies by the Over...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.odi.org/publications/10505-cash-tr...,,GT,A systematic review of 201 studies on cash tra...,,,"Economic, Health, Employment, Educational Effe...",...,,,Y,1.0,0.0,0.0,40000,1,4,4_hover
7,"Hagen-Zanker, J., Bastagli, F., Harman, L., Ba...",A systematic review of 201 studies by the Over...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.odi.org/publications/10505-cash-tr...,,GT,A systematic review of 201 studies on cash tra...,,,"Economic, Health, Employment, Educational Effe...",...,,,Y,1.0,0.0,0.0,40000,1,7,7_hover
7,"Hagen-Zanker, J., Bastagli, F., Harman, L., Ba...",A systematic review of 201 studies by the Over...,"Cash Transfers, Wealth Transfers and Unearned ...",https://www.odi.org/publications/10505-cash-tr...,,GT,A systematic review of 201 studies on cash tra...,,,"Economic, Health, Employment, Educational Effe...",...,,,Y,1.0,0.0,0.0,40000,1,8,8_hover


In [21]:
df4.shape

(56, 25)

In [22]:
# Save copy as csv
df4.to_csv('/Volumes/Samsung_t3/project-repos/jfi/cashtransfers/data/data_modified.csv', index=False)

In [23]:
data = pd.read_csv('/Volumes/Samsung_t3/project-repos/jfi/cashtransfers/data/data_modified.csv')

In [25]:
# Save copy as json
import csv
import json

csvfile = open('/Volumes/Samsung_t3/project-repos/jfi/cashtransfers/data/data_modified.csv', 'r')
jsonfile = open('/Volumes/Samsung_t3/project-repos/jfi/cashtransfers/data/data.json', 'w')

reader = csv.DictReader( csvfile)
out = json.dumps( [ row for row in reader ] )
jsonfile.write(out)

76931